In [1]:
import numpy as np
import pandas as pd
import category_encoders as ce
import keras
# from pandarallel import pandarallel
# pandarallel.initialize() # this may or may not be related to a segmentation fault error, and njobs
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier

import warnings
warnings.filterwarnings("ignores", category=DataConversionWarning)

def clean(dat): 
    assert dat.isna().sum().sum()==0
    assert all([t.name in ['int64', 'float64'] for t in dat.dtypes])
    print(dat.shape)
    a = StandardScaler().fit_transform(dat.drop('target', axis=1))
    return (pd.DataFrame(data=a, columns=dat.drop('target', axis=1).columns),
           dat.target)

X, y = clean(pd.read_csv("https://raw.githubusercontent.com/ryanleeallred/datasets/master/heart.csv"))

X.head()

Using TensorFlow backend.


(303, 14)


/home/quinn/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/quinn/anaconda3/lib/python3.6/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal
0,0.952197,0.681005,1.973123,0.763956,-0.256334,2.394438,-1.005832,0.015443,-0.696631,1.087338,-2.274579,-0.714429,-2.148873
1,-1.915313,0.681005,1.002577,-0.092738,0.072199,-0.417635,0.898962,1.633471,-0.696631,2.122573,-2.274579,-0.714429,-0.512922
2,-1.474158,-1.468418,0.032031,-0.092738,-0.816773,-0.417635,-1.005832,0.977514,-0.696631,0.310912,0.976352,-0.714429,-0.512922
3,0.180175,0.681005,0.032031,-0.663867,-0.198357,-0.417635,0.898962,1.239897,-0.696631,-0.206705,0.976352,-0.714429,-0.512922
4,0.290464,-1.468418,-0.938515,-0.663867,2.082050,-0.417635,0.898962,0.583939,1.435481,-0.379244,0.976352,-0.714429,-0.512922


In [2]:
%%time

# Important Hyperparameters
inputs = X.shape[1]
epochs = 22
batch_size = 16

def create_model(optimizer, layers_topo, loss): 
    
    # Create Model
    model = Sequential()
    model.add(Dense(inputs, activation='sigmoid', input_shape=(inputs,)))
    for k in layers_topo: 
        model.add(Dense(k, activation='relu'))
        model.add(Dense(k, activation='relu'))
        model.add(Dense(k, activation='sigmoid'))
    model.add(Dense(1))
    # Compile Model
    model.compile(optimizer=optimizer, metrics=['accuracy'], loss=loss)
    
    return model

model = KerasClassifier(build_fn=create_model, verbose=0)

batch_size = [64, 128]
epochs = [256]
losses = ['binary_crossentropy', 'mse']
optimizers = ['adam', 'SGD']
layering = [[64,32,64], [32,64,32]]

param_grid = dict(batch_size=batch_size, epochs=epochs, loss=losses, optimizer=optimizers, layers_topo=layering)

grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, iid=False, verbose=0)

grid_result = grid.fit(X, y)

Best: 0.8983606576919556 using {'batch_size': 128, 'epochs': 256, 'layers_topo': [32, 64, 32], 'loss': 'binary_crossentropy', 'optimizer': 'adam'}


In [5]:
def report(g) -> str: 
    s = ''
    u =[f"best accuracy: {grid_result.best_score_}"] + [f"\tBest {k}: {v}" for k,v in g.best_params_.items()]
    
    return '\n'.join(u)

print(report(grid_result))

best accuracy: 0.8983606576919556
	Best batch_size: 128
	Best epochs: 256
	Best layers_topo: [32, 64, 32]
	Best loss: binary_crossentropy
	Best optimizer: adam


In [3]:
'''
TerminatedWorkerError: A worker process managed by the executor was unexpectedly terminated. This could be caused by a segmentation fault while calling the function or by an excessive memory usage causing the Operating System to kill the worker. The exit codes of the workers are {EXIT(1)}

​'''

'\nTerminatedWorkerError: A worker process managed by the executor was unexpectedly terminated. This could be caused by a segmentation fault while calling the function or by an excessive memory usage causing the Operating System to kill the worker. The exit codes of the workers are {EXIT(1)}\n\n\u200b'